## Chassisml Example Notebooks
Welcome to the examples section for [Chassis.ml](https://chassis.ml), which contains notebooks that leverage Chassisml to auto-containerize models built using the most common machine learning frameworks. 

**NOTE:** Chassisml provides two key functionalities: 
1. Create a Docker container from your model code and push that container image to a Docker registry. This is the default behavior.
2. Should you pass valid Modzy credentials as optional parameters, Chassisml will take the container and upload it directly to the Modzy environment you specify. You will notice most of these notebooks deploy the model to one of the Modzy internal development environments.   

Can't find the framework you are looking for or need help? Fork this repository and open a PR, we're always interested in growing this example bank! 

The primary maintainers of Chassis also actively monitor our [Discord Server](https://discord.gg/cHpzY9yCcM), so feel free to join and ask any questions you might have. We'll be there to respond and help out promptly.

In [17]:
import os
import chassisml
import numpy as np
import getpass
import json
import pandas as pd
from io import StringIO
from fastai.tabular.all import TabularDataLoaders, RandomSplitter, TabularPandas, tabular_learner, Categorify, FillMissing, Normalize, range_of, accuracy

## Enter credentials
Dockerhub creds and Modzy API Key

In [2]:
dockerhub_user = getpass.getpass('docker hub username')
dockerhub_pass = getpass.getpass('docker hub password')
modzy_api_key = getpass.getpass('modzy api key')

docker hub username········
docker hub password········
modzy api key········


## Download and process data

In [3]:
df = pd.read_csv("./data/adult_sample/adult.csv")
df.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,salary
0,49,Private,101320,Assoc-acdm,12.0,Married-civ-spouse,NaN,Wife,White,Female,0,1902,40,United-States,>=50k
1,44,Private,236746,Masters,14.0,Divorced,Exec-managerial,Not-in-family,White,Male,10520,0,45,United-States,>=50k
2,38,Private,96185,HS-grad,NaN,Divorced,NaN,Unmarried,Black,Female,0,0,32,United-States,<50k
3,38,Self-emp-inc,112847,Prof-school,15.0,Married-civ-spouse,Prof-specialty,Husband,Asian-Pac-Islander,Male,0,0,40,United-States,>=50k
4,42,Self-emp-not-inc,82297,7th-8th,NaN,Married-civ-spouse,Other-service,Wife,Black,Female,0,0,50,United-States,<50k


In [9]:
dls = TabularDataLoaders.from_csv("./data/adult_sample/adult.csv", path=os.path.join(os.getcwd(), "data/adult_sample"), y_names="salary",
    cat_names = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race'],
    cont_names = ['age', 'fnlwgt', 'education-num'],
    procs = [Categorify, FillMissing, Normalize])

In [12]:
splits = RandomSplitter(valid_pct=0.2)(range_of(df))
to = TabularPandas(df, procs=[Categorify, FillMissing,Normalize],
                   cat_names = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race'],
                   cont_names = ['age', 'fnlwgt', 'education-num'],
                   y_names='salary',
                   splits=splits)

In [13]:
# save test subset
test_df = df.copy()
test_df.drop(['salary'], axis=1, inplace=True)
test_df[:20].to_csv("./data/sample_adult_data.csv", index=False)

In [14]:
# print out sample of fully preprocess data
to.xs.iloc[:2]

,workclass,education,marital-status,occupation,relationship,race,education-num_na,age,fnlwgt,education-num
3999,5,12,3,4,1,5,1,-0.851632,2.088600,-0.422247
7583,7,16,3,4,1,1,1,0.687601,0.509044,-0.030708


In [15]:
# rebuild dataloaders with preprocessed data
dls = to.dataloaders(bs=64)

## Train model

In [18]:
# build and train model
learn = tabular_learner(dls, metrics=accuracy)
learn.fit_one_cycle(1)

epoch,train_loss,valid_loss,accuracy,time
0,0.370123,0.364591,0.832002,00:03


In [19]:
# view results
learn.show_results()

,workclass,education,marital-status,occupation,relationship,race,education-num_na,age,fnlwgt,education-num,salary,salary_pred
0,5.0,6.0,7.0,13.0,3.0,5.0,1.0,1.347273,-0.446345,-2.379941,0.0,0.0
1,5.0,10.0,3.0,11.0,6.0,5.0,1.0,-0.631742,-0.260165,1.143908,1.0,1.0
2,6.0,13.0,3.0,13.0,1.0,5.0,1.0,0.541008,0.251182,1.535447,1.0,1.0
3,5.0,6.0,3.0,15.0,1.0,5.0,1.0,1.054085,-0.497155,-2.379941,1.0,0.0
4,5.0,12.0,1.0,9.0,5.0,5.0,1.0,0.394414,1.484547,-0.422247,0.0,0.0
5,5.0,10.0,5.0,11.0,2.0,3.0,1.0,0.174524,0.341794,1.143908,0.0,0.0
6,5.0,5.0,6.0,9.0,5.0,5.0,1.0,-0.411851,2.139335,-2.771480,0.0,0.0
7,1.0,16.0,1.0,1.0,5.0,5.0,1.0,-0.705038,-0.199236,-0.030708,0.0,0.0
8,5.0,16.0,3.0,5.0,1.0,5.0,1.0,0.174524,0.521494,-0.030708,0.0,1.0


In [20]:
row, clas, probs = learn.predict(df.iloc[1])

In [21]:
test_df = pd.read_csv("./data/sample_adult_data.csv")

In [22]:
dl = learn.dls.test_dl(test_df)

In [23]:
preds = learn.get_preds(dl=dl)[0].numpy()

In [24]:
labels = ['<50k', '>50k']

## Write process function

* Must take bytes as input
* Preprocess bytes, run inference, postprocess model output, return results

In [28]:
def process(input_bytes):
    inputs = pd.read_csv(StringIO(str(input_bytes, "utf-8")))
    dl = learn.dls.test_dl(inputs)
    preds = learn.get_preds(dl=dl)[0].numpy()
    
    inference_result = {
        "classPredictions": [
            {
                "row": i+1,
                "predictions": [
                    {"class": labels[j], "score": round(pred[j], 4)} for j in range(2)
                ]
            } for i, pred in enumerate(preds)
        ]
    }
    
    structured_output = {
        "data": {
            "result": inference_result,
            "explanation": None,
            "drift": None,
        }
    }
    
    return structured_output

## Initialize Chassis Client
We'll use this to interact with the Chassis service

In [29]:
chassis_client = chassisml.ChassisClient("http://localhost:5000")

## Create and test Chassis model
* Requires `process_fn` defined above

In [30]:
# create Chassis model
chassis_model = chassis_client.create_model(process_fn=process)

# test Chassis model locally (can pass filepath, bufferedreader, bytes, or text here):
sample_filepath = './data/sample_adult_data.csv'
results = chassis_model.test(sample_filepath)
print(results)

b'{"data":{"result":{"classPredictions":[{"row":1,"predictions":[{"class":"<50k","score":0.4300000071525574},{"class":">50k","score":0.5699999928474426}]},{"row":2,"predictions":[{"class":"<50k","score":0.4932999908924103},{"class":">50k","score":0.5066999793052673}]},{"row":3,"predictions":[{"class":"<50k","score":0.9294000267982483},{"class":">50k","score":0.0706000030040741}]},{"row":4,"predictions":[{"class":"<50k","score":0.058800000697374344},{"class":">50k","score":0.9412000179290771}]},{"row":5,"predictions":[{"class":"<50k","score":0.8169000148773193},{"class":">50k","score":0.18310000002384186}]},{"row":6,"predictions":[{"class":"<50k","score":0.9800999760627747},{"class":">50k","score":0.019899999722838402}]},{"row":7,"predictions":[{"class":"<50k","score":0.8845000267028809},{"class":">50k","score":0.11550000309944153}]},{"row":8,"predictions":[{"class":"<50k","score":0.7871999740600586},{"class":">50k","score":0.21279999613761902}]},{"row":9,"predictions":[{"class":"<50k",

In [ ]:
# test environment and model within Chassis service, must pass filepath here:

# dry run before build
test_env_result = chassis_model.test_env(sample_filepath)
print(test_env_result)

## Publish model to Modzy
Need to provide model name, model version, Dockerhub credentials, and required Modzy info

In [55]:
MODZY_URL = "https://integration.modzy.engineering/api"

response = chassis_model.publish(
    model_name="Fast AI Salary Prediction",
    model_version="0.0.1",
    registry_user=dockerhub_user,
    registry_pass=dockerhub_pass,
    modzy_api_key=modzy_api_key,
    modzy_url=MODZY_URL,
    modzy_sample_input_path=sample_filepath
)

job_id = response.get('job_id')
final_status = chassis_client.block_until_complete(job_id)

Starting build job... Ok!


In [56]:
if chassis_client.get_job_status(job_id)["result"] is not None:
    print("New model URL: {}".format(chassis_client.get_job_status(job_id)["result"]["container_url"]))
else:
    print("Chassis job failed \n\n {}".format(chassis_client.get_job_status(job_id)))

New model URL: https://integration.modzy.engineering/models/fxijqx1abk/0.0.1
